In [ ]:
import os
from ultralytics import YOLO

# change directory
ROOT = "/home/suwako/workspace/"
os.chdir(ROOT)

# dataset
ds = "data/kaist-sanit.yaml"

model = YOLO("runs/transfer/kaist-trans-4ch/weights/best.pt")  # load a pretrained model (recommended for training)

ch = 4  # number of channels
nc = 1  # number of classes
os.getcwd()


In [ ]:
# freez all layers
for param in model.model.model.parameters():
    param.requires_grad = False

# select input and output layer
layers = list(model.model.model.children())
input_layer = layers[0]
output_layer = layers[-1]

# setup input and output channels
if input_layer.conv.in_channels != ch:
    print(f"change input channels {input_layer.conv.in_channels} → {ch}")
    input_layer.conv.in_channels = ch

    for param in input_layer.parameters():
        param.requires_grad = True


# setup output channels
for l in output_layer.cv3:
    l[2].out_channels = nc

for param in output_layer.parameters():
    param.requires_grad = True


In [ ]:
# transfer learning
model.train(
    data=ds,
    epochs=30,
    project="runs/transfer",
    name=f"kaist-trans-{ch}ch",
)  # train the model
